# Generating Tables for Statistical Analysis with R

* For statistical modeling with a train and a test dataset



## 1. Load packages

In [1]:
# -*- coding: utf-8 -*-
"""
Updated 12 December 2025
Author: Sylvain Haupert
"""

from IPython import get_ipython
print(__doc__)

# Clear all the variables
get_ipython().run_line_magic('reset', '-sf')

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path 
import matplotlib.pyplot as plt
import pandas as pd 
pd.options.display.float_format = "{:,.2f}".format # display numbers with 2 decimals
import os
import sys
import numpy as np
from scipy.stats import spearmanr
import seaborn as sns
from tqdm import tqdm
from maad import util

sys.path.append(str(Path('../src')))
from stat_func import bootstrap_corr, permutation_corr

# Close all the figures (like in Matlab)
plt.close("all")

# Import configuration file
import config as cfg


Updated 12 December 2025
Author: Sylvain Haupert



## 2. Notebook init and options setting

* Select the annotation file
* Select the dataset to analyze

In [2]:
"""****************************************************************************
                                   options          
****************************************************************************"""

# Set the options
SAVE = True

# Load the configuration file
# CONFIG = cfg.load_config('config_publication.yaml')
CONFIG = cfg.load_config('config_publication_local.yaml')

# initialize the list of indices, annotations and sites
INDICES_ALL = []
ANNOTATIONS_ALL = []
SITES_ALL = []

# loop over the datasets to get the filenames of the indices and annotations and the selected sites corresponding to each dataset
for DATASET in CONFIG['datasets']:

       """***************************************************************************
                     Prepare the configuration for the dataset
       ***************************************************************************"""

       filename_indices = 'indices_'+DATASET['name']+'_BW'+str(DATASET['flim_min'])+'Hz_'+str(DATASET['flim_max'])+'Hz_'+str(CONFIG['seed_level'])+'db'+'.csv'

       INDICES_ALL     += [(filename_indices, CONFIG['save_dir'])]
       ANNOTATIONS_ALL += [(CONFIG['annotations_filename'], os.path.dirname(DATASET['path']))] 
       SITES_ALL       += DATASET['sites']

       # display the dataset name
       print(f'Dataset {DATASET["name"]} is being preparing...')       
       # display the number of sites
       print(f'The dataset {DATASET["name"]} contains {len(DATASET["sites"])} sites \n')


Dataset sapsucker_woods is being preparing...
The dataset sapsucker_woods contains 1 sites 

Dataset bialowieza is being preparing...
The dataset bialowieza contains 15 sites 

Dataset hawai is being preparing...
The dataset hawai contains 2 sites 

Dataset coffee_farms is being preparing...
The dataset coffee_farms contains 2 sites 

Dataset usa_sierra_nevada_forest is being preparing...
The dataset usa_sierra_nevada_forest contains 33 sites 

Dataset uk_sussex_countryside is being preparing...
The dataset uk_sussex_countryside contains 45 sites 

Dataset ecuador is being preparing...
The dataset ecuador contains 45 sites 

Dataset risoux is being preparing...
The dataset risoux contains 1 sites 

Dataset peru is being preparing...
The dataset peru contains 7 sites 



## 3. Preparation of the dataframe df

* Open and read dataframes with indices (variables) and with labels (ground truth)
* Clean and merge both dataframes

### 3.1 Clean, merge and save the dataframe with all the data

* Indices where previously calculated with scikit-maad package (V1.5.1) on Python.
* Manual annotations are coming from different datasets that belong to very different projects. They were formated to be readable in the same way.

> Indices and manual annotations are merged into a single dataframe. Some variables are renamed while others are dropped because we do not want to analyse them. The final dataframe is saved for further analyses, for instance in R.

In [3]:
"""****************************************************************************
                                Dataframe creation  
****************************************************************************"""
# LOAD CSV
df_indices = pd.DataFrame()
for indices_csv, indices_dir in INDICES_ALL :
    df_indices = pd.concat([df_indices, pd.read_csv(os.path.join(indices_dir,indices_csv), sep=',')], axis=0)

df_label = pd.DataFrame()
for annotations_csv, annotations_dir in ANNOTATIONS_ALL :
    # add a column dataset containing the dataset name (last  part of the path in annotations_dir)
    df_temp = pd.read_csv(os.path.join(annotations_dir,annotations_csv), sep=',')
    df_temp['dataset'] = annotations_dir.split('/')[-1]
    df_label = pd.concat([df_label, df_temp], axis=0)
    
# select a site
df_label = df_label[df_label['site'].isin(SITES_ALL)]

# add a column Filename such as in df_label
df_indices['filename'] = df_indices['file'].apply(lambda x: os.path.basename(x).split('.')[0])
df_indices.drop(['Date','file','clipping'], axis='columns', inplace=True)

# set index
df_indices.set_index('filename', inplace = True)
df_label.set_index('filename', inplace = True)

# keep several columns
df_label = df_label[[ 'date', 'site', 'LAT', 'LON', 'device_id', 'biome', 'dataset', 'species_richness']]

# Rename the column biome into habitat
df_label = df_label.rename(columns={'biome':'habitat'})

# HACK
df_label['species_richness'] = df_label['species_richness'].astype('float')

# Remove  ENRf which is the same as LEQf
try :
    df_indices.drop(['ENRf', 'audio_duration'], axis='columns', inplace=True)
except :
    pass

# Create H indice, a composite of Ht and Hf
df_indices['H'] = df_indices['Ht'] * df_indices['Hf']
# Rename some indices
df_indices = df_indices.rename(columns={'NBPEAKS':'NP'})
df_indices = df_indices.rename(columns={'BI':'BIO'})
df_indices = df_indices.rename(columns={'MED':'M'})

# transform the values into float
df_indices = df_indices.select_dtypes(include=['number']).astype('float')

# Create a new df by merging df_indices and df_label
df_train_dataset_for_stat_in_R = pd.merge(df_indices, df_label, on='filename', how='inner')

if CONFIG['remove_clipping_audio'] :
    print('The number of files after removing clipped audio')

print('The total number of files {}'.format(len(df_train_dataset_for_stat_in_R)))

# display the number of files per dataset
for dataset in CONFIG['datasets'] :
    print(f'The number of files for the dataset {dataset["name"]} is {len(df_train_dataset_for_stat_in_R[df_train_dataset_for_stat_in_R["site"].isin(dataset["sites"])])}')

# display the number of files per habitat and the number of sites per habitat
print('\n')
for habitat in df_train_dataset_for_stat_in_R['habitat'].unique() :
    print(f'The number of files for the habitat {habitat} is {len(df_train_dataset_for_stat_in_R[df_train_dataset_for_stat_in_R["habitat"] == habitat])}')
    print(f'The number of sites for the habitat {habitat} is {len(df_train_dataset_for_stat_in_R[df_train_dataset_for_stat_in_R["habitat"] == habitat]["site"].unique())} \n')

# save the dataframe
if SAVE :
    df_train_dataset_for_stat_in_R.to_csv(path_or_buf=os.path.join(CONFIG['save_dir'], 'train_dataset_for_statistical_modeling_in_R.csv'), sep=',', mode='w', header=True, index=True)


The number of files after removing clipped audio
The total number of files 4484
The number of files for the dataset sapsucker_woods is 544
The number of files for the dataset bialowieza is 540
The number of files for the dataset hawai is 357
The number of files for the dataset coffee_farms is 112
The number of files for the dataset usa_sierra_nevada_forest is 132
The number of files for the dataset uk_sussex_countryside is 1982
The number of files for the dataset ecuador is 675
The number of files for the dataset risoux is 58
The number of files for the dataset peru is 84


The number of files for the habitat Forest - Temperate is 1949
The number of sites for the habitat Forest - Temperate is 65 

The number of files for the habitat Shrubland - Subtropical-tropical high altitude is 227
The number of sites for the habitat Shrubland - Subtropical-tropical high altitude is 1 

The number of files for the habitat Forest - Subtropical-tropical moist montane is 355
The number of sites for th

## Wabad Test dataset

In [16]:
"""****************************************************************************
# -------------------          options              ---------------------------
****************************************************************************"""
SAVE = True

# Load the configuration file
# CONFIG = cfg.load_config('config_publication_wabad.yaml')
CONFIG = cfg.load_config('config_publication_local_wabad.yaml')

# initialize the list of indices, annotations and sites
INDICES_ALL = []
ANNOTATIONS_ALL = []
SITES_ALL = []

In [17]:
# loop over the datasets to get the filenames of the indices and annotations and the selected sites corresponding to each dataset
for DATASET in CONFIG['datasets']:

    """***************************************************************************
                    Prepare the configuration for the dataset
    ***************************************************************************"""

    filename_indices = 'indices_'+DATASET['name']+'_BW'+str(DATASET['flim_min'])+'Hz_'+str(DATASET['flim_max'])+'Hz_'+str(CONFIG['seed_level'])+'db'+'.csv'

    INDICES_ALL     += [(filename_indices, CONFIG['save_dir'])]
    ANNOTATIONS_ALL += [(CONFIG['annotations_filename'], os.path.dirname(DATASET['path']))] 
    SITES_ALL       += DATASET['sites']

    # display the dataset name
    print(f'Dataset {DATASET["name"]} is being preparing...')       
    # display the number of sites
    print(f'The dataset {DATASET["name"]} contains {len(DATASET["sites"])} sites \n')


"""****************************************************************************
                                Dataframe creation  
****************************************************************************"""
# LOAD CSV
df_indices = pd.DataFrame()
for indices_csv, indices_dir in INDICES_ALL :
    df_indices = pd.concat([df_indices, pd.read_csv(os.path.join(indices_dir,indices_csv), sep=',')], axis=0)

df_label = pd.DataFrame()
for annotations_csv, annotations_dir in ANNOTATIONS_ALL :
    df_label   = pd.concat([df_label, pd.read_csv(os.path.join(annotations_dir,annotations_csv), sep=',')], axis=0)

# select a site
df_label = df_label[df_label['site'].isin(SITES_ALL)]

# drop clipping rows
if CONFIG['remove_clipping_audio'] :
    df_indices = df_indices[df_indices['clipping'] == 0]

# add a column Filename such as in df_label
df_indices['filename'] = df_indices['file'].apply(lambda x: os.path.basename(x).split('.')[0])
df_indices.drop(['Date','file','clipping'], axis='columns', inplace=True)

# set index
df_indices.set_index('filename', inplace = True)
df_label.set_index('filename', inplace = True)

# keep only richness ('species_richness')
df_label = df_label[[ 'date', 'site', 'device_id', 'biome','species_richness']]

# Rename the column biome into habitat
df_label = df_label.rename(columns={'biome':'habitat'})

# HACK
df_label['species_richness'] = df_label['species_richness'].astype('float')

# Remove  ENRf which is the same as LEQf
try :
    df_indices.drop(['ENRf', 'audio_duration'], axis='columns', inplace=True)
except :
    pass

# Create H indice, a composite of Ht and Hf
df_indices['H'] = df_indices['Ht'] * df_indices['Hf']
# Rename isome indices
df_indices = df_indices.rename(columns={'NBPEAKS':'NP'})
df_indices = df_indices.rename(columns={'BI':'BIO'})
df_indices = df_indices.rename(columns={'MED':'M'})

# transform the values into float
df_indices = df_indices.select_dtypes(include=['number']).astype('float')

# Create a new df by merging df_indices and df_label
df = pd.merge(df_indices, df_label, on='filename', how='inner')

print('The total number of files {}'.format(len(df)))

# display the number of files per dataset
for dataset in CONFIG['datasets'] :
    print(f'The number of files for the dataset {dataset["name"]} is {len(df[df["site"].isin(dataset["sites"])])}')

# display the number of files per habitat and the number of sites per habitat
print('\n')
for habitat in df['habitat'].unique() :
    print(f'The number of files for the habitat {habitat} is {len(df[df["habitat"] == habitat])}')
    print(f'The number of sites for the habitat {habitat} is {len(df[df["habitat"] == habitat]["site"].unique())} \n')

Dataset wabad is being preparing...
The dataset wabad contains 70 sites 

The total number of files 1552
The number of files for the dataset wabad is 1552


The number of files for the habitat Forest - Subtropical-tropical moist montane is 208
The number of sites for the habitat Forest - Subtropical-tropical moist montane is 8 

The number of files for the habitat Shrubland - Subtropical-tropical high altitude is 34
The number of sites for the habitat Shrubland - Subtropical-tropical high altitude is 5 

The number of files for the habitat Forest - Subtropical-tropical dry is 178
The number of sites for the habitat Forest - Subtropical-tropical dry is 11 

The number of files for the habitat Forest - Subtropical-tropical moist lowland is 175
The number of sites for the habitat Forest - Subtropical-tropical moist lowland is 9 

The number of files for the habitat Shrubland - Temperate is 71
The number of sites for the habitat Shrubland - Temperate is 3 

The number of files for the habi

In [18]:
"""------------------------------------------------------------------------
Group data and compute the correlation the indice and the ground truth. 
-------------------------------------------------------------------------"""

df_save_for_stat_in_R = df.copy()

# Use the groupby method to group by the 'site' column
grouped = df_save_for_stat_in_R.groupby('site')

# Filter numeric columns
numeric_columns = df.select_dtypes(include=[int, float])

# Group by 'site' and calculate the mean for each numeric column
df_test_dataset_for_stat_in_R_count= grouped[numeric_columns.columns].count()
df_test_dataset_for_stat_in_R_mean = grouped[numeric_columns.columns].mean()

# add a count column to keep the nummber of 1min audio files by site
df_test_dataset_for_stat_in_R_mean['count'] = df_test_dataset_for_stat_in_R_count.iloc[:,0]

# save the dataframe for stat in R
if SAVE:
    new_data_path = os.path.join(CONFIG['save_dir'], 'test_dataset_for_statistical_modeling_in_R.csv')
    df_test_dataset_for_stat_in_R_mean.to_csv(new_data_path, index=True)
    print(f'The dataframe for GLMM model has been saved to {new_data_path}')

The dataframe for GLMM model has been saved to ../results/test_dataset_for_statistical_modeling_in_R.csv
